In [5]:
pwd

'c:\\Users\\hp\\Desktop\\Master - AI\\Semestre II\\Computer Vision\\CV-crowd-flow-estimation-\\notebooks'

In [ ]:
import sys
import os
# Add project root to sys.path (one directory up from the notebook)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

from models.resnet50_backbone import ResNet50Backbone
from models.vgg_backbone import VGG19BNBackbone

from src.data_loader import ShanghaiTechDataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms


In [ ]:
# ====== Part A evaluation ======

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# common transform + dataloader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
val_dataset_A = ShanghaiTechDataset("../data/ShanghaiTech", part="part_A", split="test_data", transform=transform)
val_loader_A = DataLoader(val_dataset_A, batch_size=1)

# evaluate each model
for name, Cls in [("resnet50", ResNet50Backbone),
                  ("vgg19_bn", VGG19BNBackbone),]:
    # load
    model = Cls().to(device)
    model.load_state_dict(torch.load(f"../models/part_A_{name}_model.pth", map_location=device))
    model.eval()

    mae = mse = 0.0
    with torch.no_grad():
        for img, count_map in val_loader_A:
            img = img.to(device)
            count = count_map.sum(dim=(1,2)).unsqueeze(1).float().to(device)
            out = model(img)
            mae += torch.abs(out - count).item()
            mse += ((out - count)**2).item()

    mae /= len(val_dataset_A)
    rmse = (mse / len(val_dataset_A))**0.5
    print(f"Part A — {name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}")


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EOFError: 

In [ ]:
# ====== Part B evaluation ======


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
val_dataset_B = ShanghaiTechDataset("../data/ShanghaiTech", part="part_B", split="test_data", transform=transform)
val_loader_B = DataLoader(val_dataset_B, batch_size=1)

for name, Cls in [("resnet50", ResNet50Backbone),
                  ("vgg19_bn", VGG19BNBackbone)]:
    model = Cls().to(device)
    model.load_state_dict(torch.load(f"../models/part_B_{name}_model.pth", map_location=device))
    model.eval()

    mae = mse = 0.0
    with torch.no_grad():
        for img, count_map in val_loader_B:
            img = img.to(device)
            count = count_map.sum(dim=(1,2)).unsqueeze(1).float().to(device)
            out = model(img)
            mae += torch.abs(out - count).item()
            mse += ((out - count)**2).item()

    mae /= len(val_dataset_B)
    rmse = (mse / len(val_dataset_B))**0.5
    print(f"Part B — {name}: MAE = {mae:.3f}, RMSE = {rmse:.3f}")